# cvやlbのスコアを考慮したアンサンブル
スコアをもとに加重平均をしてアンサンブルする  
スコアはファイル名に記載する　  
e.g. 0.152_submission.csv

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import copy
import glob
import random
from random import random as rd
import gc

In [2]:
df_train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
unique_pressures = df_train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        return sorted_pressures[-1]
    elif insert_idx == 0:
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

def set_seed(seed = 42):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

def blend(dp):
    # input: the dataset path of the prediction result files
    # file name format: public lb score or pulbic lb score + name, e.g., 0.335_LSTM_baseline
    # get all the files to blend
    files = []
    for file in glob.iglob(f'{dp}/*'):
        files.append(file)
    
    scores = []
    predictions = []
    for file in files:
        root, ext = os.path.splitext(file)
        basename = os.path.basename(root)
        print(basename)
        public_lb_score = int(file.split("/")[-1].split(".")[1].split("_")[0]) 
        scores.append(public_lb_score)
        predictions.append((pd.read_csv(file).pressure).ravel())
    
    for i in range(len(files)):
        if i==0:
            prediction = predictions[i] * scores[i] / sum(scores)
        else:
            prediction += predictions[i] * scores[i] / sum(scores)
    
    output_dir = './wab_submissions/'
    os.makedirs(output_dir, exist_ok=True)
    output = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
    output.pressure = prediction
    output["pressure"] = output["pressure"].apply(find_nearest)
    output.to_csv(output_dir+f'wab_submission_{input_dir}.csv',index=False)

In [3]:
input_dir='all'
set_seed(seed=42)
blend(f'./submissions/{input_dir}')

0.149_submission_median_1020_lstm_hidden=1024_6layer-ver6_mod-feat11_dropout=0.0
0.149_submission_Ventilator_LSTM_Model
0.152_submission_median_1012_lstm_hidden=1024_4layer_dropout=0.0_mod-feat5
0.153_submission_median_1016_lstm+gru
0.151_submission_median_1017_lstm_hidden=1024_6layer-ver2_mod-feat3_dropout=0.0_SELU 
0.139_Pulp _iction_median_submission
0.155_submission_median_1011_lstm_hidden=1024_4layer_dropout=0.0_mod-feat1
0.150_submission_median_1016_lstm_hidden=1024_6layer-ver2_mod-feat7_dropout=0.0_noemb
0.149_submission_median_1016_lstm_hidden=1024_6layer-ver2_mod-feat7_dropout=0.0
0.153_submission_median_1011_lstm_hidden=1024_4layer_dropout=0.0_mod-feat2
0.149_submission_median_1018_lstm_hidden=1024_6layer-ver6_mod-feat3_dropout=0.0
0.155_submission_median_1013_lstm_hidden=1024_5layer_mod-feat3_seq_emb=2layer
0.151_submission_median_1012_lstm_hidden=1024_5layer_mod-feat3
0.149_submission_median_1015_lstm_hidden=1024_6layer-ver2_mod-feat3_dropout=0.0
0.148_Random_Weights_Blendi